In [19]:
###IMPORTANT ####
 # Actor until 1976
 # Actor in a Leading Role since 1977
 # Same for Actress
    
### Stock the data better than doing a request every try # Done

### PROBLEME : COMMENT GERER LES 2 WINNERS, existe plusieurs fois # Done

### PROBLEM : for DIRECTION, names and titles are swapped # Done


### PROBLEME : Incohérence des noms de catégories sur plusieurs année : best motion picture -> best picture
# Special effect -> Visual effects etc

### Les informations récoltées ne sont pas les mêmes ex : foreign film c'est le pays qui est récolté, comment faire ?

# * bonjour
# ! alert 

import sys
!{sys.executable} -m pip install jellyfish

In [11]:
### OOP   ### ROTTENTOMATOES

import time
import requests
from scrapy import Selector
from tqdm import tqdm
import pandas as pd
import re  # text processing
import numpy as np
import json
import jellyfish
from urllib.parse import quote
import os 
import sys # to use exit()
from tkinter import * # Window 

class Oscar_Scraper:
    
    list_names = []
    list_year = []
    list_category = []
    list_films = []
    list_results = [] 
    list_ratings_imdb = []
    list_ratings_rottentomatoes = []
    list_metacritics = []
    list_boxoffice = []     
    list_id_movie = []
    list_birthday = []
    list_gender = []
    list_id_indiv = []
    list_names_right = []
    list_films_right = []
    list_names_wrong = []
    list_films_wrong = []
    
    def __init__(self, from_time, to_time):
        self.links = {
            'oscars':[],
            'imdb':[],
        }
        
        self.categories_interest = ['Directing', 'Actor', 'Actor in a Leading Role', 'Actress', 'Actress in a Leading Role',
                  'Outstanding Picture','Outstanding Production','Best Motion Picture', 'Best Picture', 'Foreign Language Film', 
                  'Special Effects', 'Special Visual Effects','Special Achievement Award (Visual Effects)', 'Visual Effects']
        
        self.categories_individuals = ['Directing', 'Actor', 'Actor in a Leading Role', 'Actress', 'Actress in a Leading Role']
        self.categories_films = ['Outstanding Picture','Outstanding Production','Best Motion Picture', 'Best Picture', 'Foreign Language Film', 
                  'Special Effects', 'Special Visual Effects','Special Achievement Award (Visual Effects)', 'Visual Effects']
        
        #lists of different names for same categories
        self.categories_dictionary = {
                    "Actor in a Leading Role": ['Actor', 'Actor in a Leading Role'],
                    "Actress in a Leading Role": ['Actress', 'Actress in a Leading Role'],
                    "Directing": ["Directing"],
                    "Best Picture": ['Outstanding Picture', 'Outstanding Production','Best Motion Picture', 'Best Picture'],
                    "Visual Effects" : ['Special Effects', 'Special Visual Effects','Special Achievement Award (Visual Effects)','Visual Effects']
                    }
        
        self.selected_categories = []
        

        try:
            self.from_time = int(from_time)
            self.to_time = int(to_time)
        except Exception as e:
            print("You must enter a valid date.\nThe object was not created.")
        
        """ list_imput_category = []
        blabla = ""
        while blabla != "Stop":
            blabla = input("Write one category you want to search then press enter. When you're done, write 'Stop'")
            if blabla != "Stop":
                list_imput_category.append(blabla)
        for imput_category_index in range(len(list_imput_category)):
            if list_imput_category[imput_category_index] not in self.categories_interest:
                print("You must enter a valid category among these one:", self.categories_interest)
                sys.exit()
        """
        self.time_period = range(self.from_time, self.to_time + 1) 

        self.data = {
                'category':[],
                'film':[],
                'year':[],
                'name':[],
                'gender':[],
                'result':[],
                'birthdate':[],

    """         'imbd_ratings':[],
                'rotten_ratings':[],
                'metacritics_ratings':[],
                'box office':[], 
    """
                'id_movie':[]
                
            }

    def Imput(self):
        window = Tk()
        window.title('Which categories do you want to compile ?')
        
        # for scrolling vertically
        yscrollbar = Scrollbar(window)
        yscrollbar.pack(side = RIGHT, fill = Y)
        
        label = Label(window,
                    text = "Which categories do you want to compile ?\n Select the categories below :  ",
                    font = ("Times New Roman", 10), 
                    padx = 10, pady = 10)
        label.pack()
        list_selection = Listbox(window, selectmode = "multiple", 
                    yscrollcommand = yscrollbar.set)
        
        # Widget expands horizontally and 
        # vertically by assigning both to
        # fill option
        list_selection.pack(padx = 10, pady = 10,
                expand = YES, fill = "both")
        
        x = [item for item in list(self.categories_dictionary.keys())]


        for each_item in range(len(x)):
            
            list_selection.insert(END, x[each_item])
            list_selection.itemconfig(each_item, bg = "white")

        exit_button = Button(window, text="Done", command= lambda:[done_button(),window.destroy()])
        exit_button.pack(pady=20)
        
        # Attach listbox to vertical scrollbar
        yscrollbar.config(command = list_selection.yview)

        def done_button():
            self.input_selected = []
            for i in list_selection.curselection():
                self.input_selected.append(list_selection.get(i)) 
            

        window.mainloop()

        for input_categories in self.input_selected:
            if input_categories in self.categories_dictionary.get(input_categories):
                for number_name in range(len(self.categories_dictionary.get(input_categories))):
                    self.selected_categories.append(self.categories_dictionary.get(input_categories)[number_name])
        print(self.selected_categories)
 
                

    def getHTML(self):

        """"
        Main method.
        Used to extract the content of the oscar.org webpage for each ceremonies included in the time period chose by the utilisator.
        It stocks them in self.links['oscars']
        Returns nothing
        """
        
        for number_year in tqdm(self.time_period):
            #time.sleep(10) # Crawlerdelay
            url = "https://www.oscars.org/oscars/ceremonies/{}".format(number_year)
            response_oscars = requests.get(url)
            sel = Selector(text = response_oscars.text)   # Scrapy chose over BeautifulSoup for selector CSS
            self.links['oscars'].append(sel)

    def getNominees(self, years, i, category, number_nominees, number_winners):

        """
        Sub-method.
        It takes itself, the index of which year it is, the index of which category it is, the category, the number of nominees and the number of winners
        It extracts and stock the name of the nominees and the film nominated
        Returns nothing.
        """

        for number_people_nominated in range(number_nominees):   
            if number_winners == 1:
                # NAME
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 4 + number_people_nominated)
                nominee_name = ''.join(self.links['oscars'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
            if number_winners == 2:
                 # NAME
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 5 + number_people_nominated)
                nominee_name = ''.join(self.links['oscars'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
            
            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)

            # FILM
            if number_winners == 1:
                path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 4 + number_people_nominated)
                nominee_film = ''.join(self.links['oscars'][years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_film = nominee_film.replace("\n", "") # Cleaning up the string
            if number_winners == 2:
                path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 5 + number_people_nominated)
                nominee_film = ''.join(self.links['oscars'][years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_film = nominee_film.replace("\n", "") # Cleaning up the string
            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_names.append(nominee_film)
            else:
                self.list_films.append(nominee_film)
            
            self.list_year.append(self.from_time + years)         
            self.list_category.append(category)
            self.list_results.append("Nominee")

    def getWinners(self, years, i, category, number_winners):
        """
        Sub-method.
        It takes itself, the index of which year it is, the index of which category it is, the category and the number of winners
        It extracts and stock the name of the winners and the film winning.
        Returns nothing.       
        """

        if number_winners == 1: # 1 Winner
        # NAME
            path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
            nominee_name = ''.join(self.links['oscars'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string

            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)
            
        
            # FIlM
            path_film = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-title > span::text'.format(i+1)
            nominee_film = ''.join(self.links['oscars'][years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
            nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(nominee_film)
            else:
                self.list_films.append(nominee_film)
           
            self.list_year.append(self.from_time + years)
            self.list_category.append(category)
            self.list_results.append("Winner")


        if number_winners == 2: # 2 Winners
            # NAME 1st Winner 
            path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
            nominee_name = ''.join(self.links['oscars'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4

            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)

            self.list_year.append(self.from_time + years)
            self.list_category.append(category)
            self.list_results.append("Tie-winner")

            # NAME 2st Winner 
            path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
            nominee_name = ''.join(self.links['oscars'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4

            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)

            self.list_year.append(self.from_time + years)
            self.list_category.append(category)
            self.list_results.append("Tie-winner")

            # FILM
            path_film1 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-title > span::text'.format(i+1)
            path_film2 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-title > span::text'.format(i+1)

            nominee_film = ''.join(self.links['oscars'][years].css(path_film1).extract()) # ''.join() transforms the list produced by the selector into a string
            nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(nominee_film)
            else:
                self.list_films.append(nominee_film)

            winner2_film = ''.join(self.links['oscars'][years].css(path_film2).extract()) # ''.join() transforms the list produced by the selector into a string
            winner2_film = winner2_film.replace("\n", "") # Cleaning up the string

            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(winner2_film)
            else:
                self.list_films.append(winner2_film)        



    def getDATA(self): 

        """
        Main method:
        Using the content of each oscar.org webpage per year to identify the number of winners and nominated of each category in the list, then through two sub-methods it identifies the result, year, name of the winner/nominee,
        title of the movie and category, and save them in a dictionnary of list.
        Then it calls a sub-method to proceed some corrections on these lists, as some errors exist in the oscar.org website.
        Returns nothing
        """
            
        for years in tqdm(range(len(self.links['oscars']))):

            categories = self.links['oscars'][years].css('#quicktabs-tabpage-honorees-0 > div > div.view-content > div.view-grouping > div.view-grouping-header > h2::text').extract()    
            
            for i in range(len(categories)):
                if categories[i] in self.selected_categories:

                    # Need the number of winners (might be ties) and nominated          
                    path_nominated = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:last-child".format(i+1)
                    number_of_film_nominated = int(''.join(self.links['oscars'][years].css(path_nominated).extract()).split("views-row-")[1])

                    path_winner = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2)".format(i+1)
                    trial = ''.join(self.links['oscars'][years].css(path_winner).extract()).split("views-row-first views-row-last") # If 1 winner then it splits the string into 1 list of 2 elements, if 2 winners, it does nothing
                    if len(trial) == 2:
                        number_winners = 1
                    elif len(trial) == 1:
                        number_winners = 2
                    else:
                        print("problem")


                    ## WINNERS                 
                    self.getWinners(years, i, categories[i], number_winners)     
                    ## NOMINEES
                    self.getNominees(years, i, categories[i], number_of_film_nominated, number_winners)   
                    
                    
        self.data = {
            'year': self.list_year,
            'category': self.list_category,
            'film': self.list_films,
            'name': self.list_names,
            #'gender':[],
            'result':self.list_results
            #,
            #'birthdate':[]
        }            
    
        self.Correction(0,True)
        

    def getINDIVIDUALS(self):
        df = pd.DataFrame(self.data)
        df = df.set_index(['year','category'])
        individuals = df.loc(axis = 0)[pd.IndexSlice[:, self.categories_individuals]]
        return(individuals)
    
    def saveINDIVIDUALS(self):
        df = pd.DataFrame(self.data)
        df = df.set_index(['year','category'])
        individuals = df.loc(axis = 0)[pd.IndexSlice[:, self.categories_individuals]]
        individuals.reset_index() # For some reasons the index is not saved with the rest so we make it back to two columns 
        individuals.to_csv("Individuals.csv")
        return("The dataframe was saved as Individuals.csv in your working directory.")
        
    def getFILMS(self):
        df = pd.DataFrame(self.data)
        df = df.set_index(['year','category'])
        films = df.loc(axis = 0)[pd.IndexSlice[:, self.categories_interest[5:]]]
        films.columns = ('film', 'Studio/Creator(s)', 'result')
        return(films)
    
     #creating big dataframe with all data
    def getAllCategories(self):
        df = pd.DataFrame(self.data)
        df = df.set_index(['year','category'])
        return(df)
    
    #in progress
    """ #creating CSV files based on scraped data. Possible naming of the columns for different datasets
    def create_named_CSV(self,dataframe,categories,columns_name):
        if len(column_name) > 2:
          raise Exception('Items exceeds the maximum allowed length of 2')
        category = []
        if type(categories) == 'str':
        category.append(categories)
        for x in self.categories_dictionary:
            for y in range(len(self.categories_dictionary[x])):
                if category == self.categories_dictionary[x][y]:
                    if x not in names:
                        names.append(x)
        for x in names:
            create_empty_CSV(x)
        df_final.columns = columns_name
             films.to_csv(i + '.csv', sep=' ', mode='a')"""
        
    #creating empty csv names after argument of the method   
    def create_empty_CSV(*stringcsv_names):
        for i in stringcsv_names:
            with open(i + '.csv', 'w') as f:
                pass
            
######################################################### 
    
    """      
    def getAPI_omdb(self):
        key = "59d5666b"
        url_api = "https://www.omdbapi.com/?t={}&y={}&apikey={}"
        for film_number in tqdm(range(len(self.data['film']))):
            film = self.data['film']
            title_standard = film[film_number].upper().replace(" ","+").replace(".","").replace("&","%26").replace("ONCE+UPON+A+TIMEIN+HOLLYWOOD", "ONCE+UPON+A+TIME...+IN+HOLLYWOOD")
            year_standard = self.data['year'][film_number]-1
            response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
            if response_api["Response"] != "False":        
                self.Ombd_get(response_api)
            else:
                year_standard = self.data['year'][film_number]-2
                response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                if response_api["Response"] != "False":               
                    self.Ombd_get(response_api)
                else:
                    year_standard = self.data['year'][film_number]-3
                    response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                    if response_api["Response"] != "False":
                        self.Ombd_get(response_api)
                    else:
                        year_standard = self.data['year'][film_number]
                        response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                        if response_api["Response"] != "False":
                            self.Ombd_get(response_api)
                        else:
                            title_standard = title_standard.replace("%26","and")
                            year_standard = self.data['year'][film_number]-1
                            response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                            if response_api["Response"] != "False":
                                self.Ombd_get(response_api)
                            else:
                                year_standard = self.data['year'][film_number]-2
                                response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                                if response_api["Response"] != "False":
                                    self.Ombd_get(response_api)
                                else: 
                                    year_standard = self.data['year'][film_number]-3
                                    response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                                    if response_api["Response"] != "False":
                                        self.Ombd_get(response_api)
                                    else:
                                        print("Error",film[film_number],title_standard,year_standard)
            
        # Standardization
        
        for i in tqdm(range(len(self.data['film']))): 
            if self.list_boxoffice[i] == "N/A":
                self.list_boxoffice[i] = np.nan
            else:
                self.list_boxoffice[i] = float(self.list_boxoffice[i].replace("$","").replace(",",""))

            if self.list_ratings_imdb[i] == "N/A":
                self.list_ratings_imdb[i] = np.nan
            else:
                self.list_ratings_imdb[i] = float(self.list_ratings_imdb[i])

            if self.list_ratings_rottentomatoes[i] == "N/A":
                self.list_ratings_rottentomatoes[i] = np.nan
            else:
                self.list_ratings_rottentomatoes[i] = float(self.list_ratings_rottentomatoes[i].replace("%",""))

            if self.list_metacritics[i] == "N/A":
                self.list_metacritics[i] = np.nan
            else:
                self.list_metacritics[i] = float(self.list_metacritics[i].replace("/100",""))
            if self.data["film"][i] == "The Postman (Il Postino)":
                self.list_id_movie[i] = "tt0110877"
    
        self.data['imbd_ratings'] = self.list_ratings_imdb,
        self.data['rotten_ratings'] = self.list_ratings_rottentomatoes,
        self.data['metacritics_ratings'] = self.list_metacritics,
        self.data['box office'] = self.list_boxoffice
        self.data['id_movie'] = self.list_id_movie
        
        
        
    def Ombd_get(self, response_api):
        for ratings in range(len(response_api["Ratings"])):

            if response_api["Ratings"][ratings]["Source"] == "Rotten Tomatoes":
                self.list_ratings_rottentomatoes.append(response_api["Ratings"][ratings]["Value"])

            if response_api["Ratings"][ratings]["Source"] == "Metacritic":
                self.list_metacritics.append(response_api["Ratings"][ratings]["Value"]) 
                    
        if not any("Rotten Tomatoes" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
            self.list_ratings_rottentomatoes.append("N/A")
        if not any("Metacritic" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
            self.list_metacritics.append("N/A")

        self.list_ratings_imdb.append(response_api.get("imdbRating", "N/A"))            
        self.list_boxoffice.append(response_api.get("BoxOffice", "N/A"))
        self.list_id_movie.append(response_api.get("imdbID", "N/A"))   

    """
        
        
    def getAPI_TMDB(self):

        """
        Main method:

        From the title, it searches in an complete movie database for the movie, then we extract the matching individual and collect their birthdate and gender.
        It uses the sub-method getTMDB
        Returns nothing
        
        """
        API_KEY_MDB = "a68690ebf69567801e68c26ee82d7787"
        
        URL_MDB_SEARCH = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}&page={}&include_adult=false"
        URL_MDB_PERSON = "https://api.themoviedb.org/3/person/{}?api_key={}"

        for film_number in tqdm(range(len(self.data['film']))):
            title_standard = quote(self.data['film'][film_number])
            response_search = json.loads(requests.get(URL_MDB_SEARCH.format(API_KEY_MDB,title_standard,1)).text)
            if response_search["total_results"] != 0:        
                if self.TMDB_get(response_search, film_number) == True:

                    response_person = json.loads(requests.get(URL_MDB_PERSON.format(self.list_id_indiv[film_number],API_KEY_MDB)).text)
                    if response_person.get("success") != False: # Check if the API find a person

                        if response_person["birthday"] in (None,0):
                            pass
                        
                        if response_person["gender"] in (None,0):
                            pass
                        self.list_birthday.append(response_person["birthday"])
                        self.list_gender.append(response_person["gender"])
                    else:
                        pass
                else:
                    response_search = json.loads(requests.get(URL_MDB_SEARCH.format(API_KEY_MDB,title_standard,2)).text)
                    if response_search["total_results"] != 0:        
                        if self.TMDB_get(response_search, film_number) == True:
                            response_person = json.loads(requests.get(URL_MDB_PERSON.format(self.list_id_indiv[film_number],API_KEY_MDB)).text)
                            if response_person.get("success") != False: # Check if the API find a person
                                if response_person["birthday"] in (None,0):
                                    pass
                                
                                if response_person["gender"] in (None,0):
                                    pass
                                self.list_birthday.append(response_person["birthday"])
                                self.list_gender.append(response_person["gender"])
                            else:
                                pass
                        else:
                            # specific problems with some 1930's movies
                            URL_MDB_SEARCH1930 = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}&page={}&include_adult=false&year=1930"
                            response_search = json.loads(requests.get(URL_MDB_SEARCH1930.format(API_KEY_MDB,title_standard,1)).text)
                            if response_search["total_results"] != 0:        
                                if self.TMDB_get(response_search, film_number) == True:
                                    response_person = json.loads(requests.get(URL_MDB_PERSON.format(self.list_id_indiv[film_number],API_KEY_MDB)).text)
                                    if response_person.get("success") != False: # Check if the API find a person
                                        
                                        if response_person["birthday"] in (None,0):
                                            pass
                                        if response_person["gender"] in (None,0):
                                            pass
                                        self.list_birthday.append(response_person["birthday"])
                                        self.list_gender.append(response_person["gender"])
                                    else:
                                        pass
            else:
                pass
                self.Correction(film_number)
        
        for i in tqdm(range(len(self.data['film']))): 
            if self.list_gender[i] == 1:
                self.list_gender[i] = "Female"
            elif self.list_gender[i] == 2:
                self.list_gender[i] = "Male"
            elif self.list_gender[i] == 3:
                self.list_gender[i] = "Non-binary"
##
            self.list_birthday[i] = int(self.list_birthday[i][0:4]) # We keep only the year and transform from string to float
            
        self.data['gender'] = self.list_gender
        self.data['birthday'] = self.list_birthday
        
        
                                        
    def TMDB_get(self, response_search, film_number):
        """
        Sub-method.
        It takes itself, the content of the response of the API, the index number of the film in self.data["film"].
        Used by getAPI_tmdb to extract to confirm the right movie was picked (it contains the individual it is looking for)
        Returns True or False, True if the individual was present in the credits of the movie.
        """
        
        url_MDB_credit = "http://api.tmdb.org/3/movie/{}/credits?api_key={}"
        API_KEY_MDB = "a68690ebf69567801e68c26ee82d7787"
        found_individual = False
        
        for number_results in range(len(response_search["results"])):
            if found_individual == False:
                if response_search["results"][number_results].get("release_date") not in (None,0,''): # There is a release date
                    if (int(response_search["results"][number_results]["release_date"][0:4]) in (self.data["year"][film_number],self.data["year"][film_number]-1, self.data["year"][film_number]-2, self.data["year"][film_number]-3)) and found_individual == False:
                        id_MDB = response_search["results"][number_results]["id"]
                        response_credit = json.loads(requests.get(url_MDB_credit.format(id_MDB,API_KEY_MDB)).text)
                        if response_credit.get("success") != False: # Check if the API find a person
                            if self.data["category"][film_number] == "Directing":
                                for acteurs in range(len(response_credit["crew"])):
                                    if jellyfish.damerau_levenshtein_distance(response_credit["crew"][acteurs]["name"], self.data["name"][film_number]) < 2:
                                        self.list_id_indiv.append(response_credit["crew"][acteurs]["id"])
                                        found_individual = True
                                       
                                        break


                                if not any(jellyfish.damerau_levenshtein_distance(response_credit["crew"][acteurs]["name"], self.data["name"][film_number]) < 2 for acteurs in range(len(response_credit["crew"]))):
                                    pass
                                    # self.Correction(film_number, False)
                                    
                            else:
                                for acteurs in range(len(response_credit["cast"])):
                                    if jellyfish.damerau_levenshtein_distance(response_credit["cast"][acteurs]["name"], self.data["name"][film_number]) < 2:
                                        self.list_id_indiv.append(response_credit["cast"][acteurs]["id"])
                                        found_individual = True  # Empêche d'avoir plusieurs fois le même acteur si on l'a déjà trouvé
                                        break   
                                        
                                if not any(jellyfish.damerau_levenshtein_distance(response_credit["cast"][acteurs]["name"], self.data["name"][film_number]) < 2 for acteurs in range(len(response_credit["cast"]))):
                                    pass
                                    # self.Correction(film_number, False)
                                    

                        else:
                            pass
                    else:
                        pass

                else:
                    pass
        return(found_individual)
                                

    def Correction(self,film_number, corrected):
        """
        Sub-method.
        Used in two ways:
        1) It was used to manually identify and correct mistakes hidden in the oscar.org website, stocking them in a dataframe to use at each iteration of the script. It has been disabled once every mistake was corrected.
        2) It loads the dataframes precedently created, then correct them 
        Returns nothing.
        """
        if corrected == False:
            question = input("Film (f) or Name (n) or Both (b) or Pass (p)?")
            if question == "f":
                answer = input("Bon titre film ?")
                self.list_films_wrong.append(self.data["film"][film_number])
                self.list_films_right.append(answer)
                self.data["film"][film_number] = answer
                pd.DataFrame(self.list_films_right).to_csv('list_films_right.csv',index=False) 
                pd.DataFrame(self.list_films_wrong).to_csv('list_films_wrong.csv',index=False) 
            if question == "n":
                answer = input("Bon nom ?")
                self.list_names_wrong.append(self.data["name"][film_number])
                self.list_names_right.append(answer)           
                self.data["name"][film_number] = answer
                pd.DataFrame(self.list_names_right).to_csv('list_names_right.csv',index=False) 
                pd.DataFrame(self.list_names_wrong).to_csv('list_names_wrong.csv',index=False) 
            if question == "b":
                answer1 = input("Bon titre film ?")
                answer2 = input("Bon nom ?")
                self.list_films_wrong.append(self.data["film"][film_number])
                self.list_names_wrong.append(self.data["name"][film_number])
                self.list_films_right.append(answer1)
                self.list_names_right.append(answer2)
                self.data["film"][film_number] = answer1
                self.data["name"][film_number] = answer2
                pd.DataFrame(self.list_names_right).to_csv('list_names_right.csv',index=False) 
                pd.DataFrame(self.list_names_wrong).to_csv('list_names_wrong.csv',index=False)
                pd.DataFrame(self.list_films_right).to_csv('list_films_right.csv',index=False) 
                pd.DataFrame(self.list_films_wrong).to_csv('list_films_wrong.csv',index=False) 
            if question == "p":
                pass

        if corrected == True:
            list_films_right = pd.read_csv("list_films_right.csv")
            list_films_wrong = pd.read_csv("list_films_wrong.csv")
            list_names_right = pd.read_csv("list_names_right.csv")
            list_names_wrong = pd.read_csv("list_names_wrong.csv")
            l_films = len(list_films_right)
            l_names = len(list_names_right)
            l_data = len(self.data["film"])
            
            for i in range(max(l_films,l_names)):
                if i < l_names:
                    self.list_names_right.append(list_names_right.values.tolist()[i][0])
                    self.list_names_wrong.append(list_names_wrong.values.tolist()[i][0])
                if i < l_films:
                    self.list_films_right.append(list_films_right.values.tolist()[i][0])
                    self.list_films_wrong.append(list_films_wrong.values.tolist()[i][0])
                    
            for i in range(l_data):
                for y in range(max(l_films,l_names)):
                    if y < l_films:
                        if self.data['film'][i] == self.list_films_wrong[y]:
                            self.data['film'][i] = self.list_films_right[y]
                    if y < l_names:       
                        if self.data['name'][i] == self.list_names_wrong[y]:
                            self.data['name'][i] = self.list_names_right[y]

            


In [12]:
test = Oscar_Scraper(1930,1940)
test.Imput()
test.getHTML()
test.getDATA()
test.getAPI_TMDB()
test.getINDIVIDUALS()


  9%|▉         | 1/11 [00:00<00:01,  8.02it/s]

['Actor', 'Actor in a Leading Role', 'Actress', 'Actress in a Leading Role', 'Directing']


  0%|          | 0/157 [00:00<?, ?it/s]

{'page': 1, 'results': [{'adult': False, 'backdrop_path': '/fGW3VitkBwhvYjfXedbMh8tpNSC.jpg', 'genre_ids': [28, 37], 'id': 83989, 'original_language': 'en', 'original_title': 'In Old Arizona', 'overview': "Army Sergeant Mickey Dunn sets out in pursuit of the Cisco Kid, a notorious if kind-hearted and charismatic bandit of the Old West. The Kid spends much of his loot on Tonia, the woman he loves, not realizing that she is being unfaithful to him in his absence. Soon, with her oblivious paramour off plying his trade, Tonia falls in with Dunn, drawn by the allure of a substantial reward for the Kid's capture -- dead or alive. Together, they concoct a plan to ambush and do away with the Cisco Kid once and for all.", 'popularity': 2.731, 'poster_path': '/x1XSQeLUdoi72P5UOXqAyL46B9a.jpg', 'release_date': '1928-12-25', 'title': 'In Old Arizona', 'video': False, 'vote_average': 5.3, 'vote_count': 13}], 'total_pages': 1, 'total_results': 1} In%20Old%20Arizona In Old Arizona


  1%|          | 1/157 [00:00<01:16,  2.03it/s]

...

{'page': 1, 'results': [{'adult': False, 'backdrop_path': '/1uOpPfARvbEf1TQfRGadWHhQqJH.jpg', 'genre_ids': [35, 18], 'id': 3083, 'original_language': 'en', 'original_title': 'Mr. Smith Goes to Washington', 'overview': "Naive and idealistic Jefferson Smith, leader of the Boy Rangers, is appointed to the United States Senate by the puppet governor of his state. He soon discovers, upon going to Washington, many shortcomings of the political process as his earnest goal of a national boys' camp leads to a conflict with the state political boss.", 'popularity': 9.065, 'poster_path': '/qZfL7UDj6KtkvAAQ5sv0v8ITKyM.jpg', 'release_date': '1939-10-19', 'title': 'Mr. Smith Goes to Washington', 'video': False, 'vote_average': 8, 'vote_count': 700}], 'total_pages': 1, 'total_results': 1} Mr.%20Smith%20Goes%20to%20Washington Mr. Smith Goes to Washington


 99%|█████████▉| 156/157 [00:57<00:00,  3.89it/s]

{'page': 1, 'results': [{'adult': False, 'backdrop_path': '/7SYNEG50fKODsTP6h9phYM3BueV.jpg', 'genre_ids': [37, 12, 28], 'id': 995, 'original_language': 'en', 'original_title': 'Stagecoach', 'overview': 'A group of people traveling on a stagecoach find their journey complicated by the threat of Geronimo, and learn something about each other in the process.', 'popularity': 7.557, 'poster_path': '/b4RUzWOalyPbUu66TT147b5iR0M.jpg', 'release_date': '1939-03-03', 'title': 'Stagecoach', 'video': False, 'vote_average': 7.7, 'vote_count': 727}, {'adult': False, 'backdrop_path': '/284a8fLy11nAs7IsvNIezySqSjx.jpg', 'genre_ids': [37], 'id': 39287, 'original_language': 'en', 'original_title': 'Stagecoach', 'overview': "A group of unlikely travelling companions find themselves on the same stagecoach to Cheyenne. They include a drunken doctor, a bar girl who's been thrown out of town, a professional gambler, a travelling liquor salesman, a banker who has decided to embezzle money, a gun-slinger out 

100%|██████████| 157/157 [00:00<?, ?it/s]


film             name   result gender  \
year category                                                                   
1930 Actor                    In Old Arizona    Warner Baxter   Winner   Male   
     Actor                       Thunderbolt  George Bancroft  Nominee   Male   
     Actor                             Alibi   Chester Morris  Nominee   Male   
     Actor                       The Valiant        Paul Muni  Nominee   Male   
     Actor                       The Patriot      Lewis Stone  Nominee   Male   
...                                      ...              ...      ...    ...   
1940 Directing            Gone with the Wind   Victor Fleming   Winner   Male   
     Directing            Goodbye, Mr. Chips         Sam Wood  Nominee   Male   
     Directing  Mr. Smith Goes to Washington      Frank Capra  Nominee   Male   
     Directing                    Stagecoach        John Ford  Nominee   Male   
     Directing             Wuthering Heights    William Wyler  Nominee   Male   

                birthday  
year category             
1930 Actor          1889  
     Actor          1882  
     Actor          1901  
     Actor          1895  
     Actor          1879  
...                  ...  
1940 Directing      1889  
     Directing      1883  
     Directing      1897  
     Directing      1894  
     Directing      1902  

[157 rows x 5 columns]

In [54]:
car = {
  "brand": ["Ford","Blabla"],
  "model": "Mustang",
  "year": 1964
}

x = list(car.keys())
x[0]


'brand'

In [27]:

test.saveINDIVIDUALS()


'The dataframe was saved as Individuals.csv in your working directory.'

In [20]:
for i in tqdm(range(len(test.data['film']))): 
            if test.list_gender[i] == 1:
                test.list_gender[i] = "Female"
            elif test.list_gender[i] == 2:
                test.list_gender[i] = "Male"
            else:
                pass
##
            test.list_birthday[i] = int(test.list_birthday[i][0:4]) # We keep only the year and transform from string to float

100%|██████████| 1372/1372 [00:00<00:00, 457112.17it/s]


In [21]:
test.list_birthday

[1889,
 1882,
 1901,
 1895,
 1879,
 1892,
 1892,
 1897,
 1890,
 1894,
 1898,
 1886,
 1878,
 1888,
 1888,
 1886,
 1886,
 1892,
 1868,
 1868,
 1885,
 1888,
 1888,
 1891,
 1891,
 1896,
 1902,
 1903,
 1892,
 1905,
 1905,
 1902,
 1899,
 1895,
 1890,
 1890,
 1889,
 1892,
 1894,
 1878,
 1922,
 1893,
 1897,
 1890,
 1868,
 1901,
 1898,
 1901,
 1902,
 1899,
 1889,
 1890,
 1895,
 1894,
 1885,
 1897,
 1892,
 1900,
 1868,
 1887,
 1894,
 1894,
 1894,
 1899,
 1893,
 1895,
 1907,
 1858,
 1906,
 1886,
 1897,
 1899,
 1901,
 1890,
 1892,
 1903,
 1908,
 1898,
 1902,
 1897,
 1888,
 1889,
 1886,
 1901,
 1899,
 1895,
 1905,
 1908,
 1897,
 1903,
 1907,
 1902,
 1911,
 1894,
 1886,
 1898,
 1886,
 1895,
 1901,
 1883,
 1892,
 1900,
 1910,
 1898,
 1900,
 1908,
 1902,
 1897,
 1902,
 1889,
 1892,
 1889,
 1900,
 1899,
 1897,
 1904,
 1895,
 1910,
 1898,
 1905,
 1906,
 1907,
 1896,
 1893,
 1893,
 1892,
 1896,
 1900,
 1899,
 1899,
 1905,
 1893,
 1908,
 1891,
 1912,
 1902,
 1909,
 1897,
 1886,
 1899,
 1894,
 1886,
 1905,